In [6]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from math import floor
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)


In [7]:
# hyperparameters for tuning: #of neurons, activation function, optimizer, learning rate, batch size, epochs, # of layers
# #of neurons;10-100 # of epochs:20-100
# neurons = [10, 32, 64]
# epochs: [1,5,10]
# batch_size: [500, 1000, 2000]
# activation: ['relu', 'tanh', 'sigmoid']
# optimizer: ['SGD', 'RMSprop', 'Adam']
# learning_rate: [0.001, 0.01, 0.1]
# momentum: [0.1, 0.3, 0.5, 0.7, 0.9]
# drop-out: [0.0, 0.2, 0.4]


In [8]:
from EDA import get_train, get_test
x_train, y_train = get_train()
x_test = get_test()
select_index = list(range(10))
x_train = x_train.iloc[:, select_index]
x_test = x_test.iloc[:, select_index]
print(x_train.shape, y_train.shape, x_test.shape)
# select_index


(19168, 10) (19168,) (6966, 10)


In [9]:
def define_model(neurons=10, activation='relu', optimizer='SGD', learning_rate=0.1, dropout=0, batch_size=500, epochs=1, init_weights='normal'):
    model = Sequential()
    model.add(Dense(neurons, input_dim=x_train.shape[1], activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout))    
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout)) 
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout)) 
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dropout(dropout)) 
    model.add(Dense(neurons, activation=activation, kernel_initializer=init_weights))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [10]:
seed = 8
np.random.seed(seed)
estimator = KerasRegressor(build_fn=define_model, verbose=0)

In [11]:
# Set paramaters
params_nn ={
    'neurons': [64, 128, 256],
    'activation': ['relu', 'tanh', 'sigmoid'],
    'optimizer': ['SGD', 'RMSprop', 'Adam'],
    'learning_rate': [0.01, 0.05, 0.1],
    'dropout': [0.1, 0.2, 0.3, 0.4, 0.5],
    'batch_size': [200,500,1000],
    'epochs': [5,10,15],
    'init_weights': ['uniform', 'normal', 'he_uniform'],
}
# Run Bayesian Optimization

grid_result = RandomizedSearchCV(
    estimator=estimator, 
    param_distributions=params_nn, 
    n_iter=300,
    cv=5, 
    n_jobs=-1,
    verbose = 10,
    scoring='neg_mean_squared_error'
)

In [12]:
grid_result.fit(x_train, y_train)

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


RandomizedSearchCV(cv=5,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x000002D51E02AD68>,
                   n_iter=500, n_jobs=-1,
                   param_distributions={'activation': ['relu', 'tanh',
                                                       'sigmoid'],
                                        'batch_size': [10, 50, 100, 200],
                                        'dropout': [0.0, 0.2, 0.4],
                                        'epochs': [5, 10, 15],
                                        'init_weights': ['uniform', 'normal',
                                                         'he_uniform'],
                                        'learning_rate': [0.001, 0.01, 0.1],
                                        'neurons': [10, 32, 64],
                                        'optimizer': ['SGD', 'RMSprop',
                                                      'Adam']},
                   scoring='neg_mean_squared_error', verbo

In [13]:
y = grid_result.predict(x_test)

In [14]:
def upload(y):
    submission = pd.read_csv('data/example-submission.csv')
    submission['Predicted'] = y
    submission.to_csv('DNN/submission5.csv', index=False)
    print(submission.head())

In [15]:
upload(y)

   Id    Predicted
0   0  1377507.125
1   1  1892688.250
2   2  1358840.125
3   3   827686.625
4   4   827686.625


In [16]:
grid_result.best_params_

{'optimizer': 'RMSprop',
 'neurons': 64,
 'learning_rate': 0.001,
 'init_weights': 'normal',
 'epochs': 15,
 'dropout': 0.2,
 'batch_size': 50,
 'activation': 'relu'}

In [29]:
y

array([1343186.2 , 1656916.6 , 1414450.1 , ..., 2478964.5 ,  492655.34,
       3266319.5 ], dtype=float32)